In [ ]:
import os
import subprocess
import sys
from typing import List
from pyspark.sql import SparkSession
import logging

# Configure logging
logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)

# Define the base directory
jars_home = '/workspace/delta-jars'

# Required core JARs
jars_list = [
    # Delta Lake
    f"{jars_home}/delta-spark_2.12-3.3.0.jar",
    f"{jars_home}/delta-storage-3.3.0.jar",
    # AWS
    f"{jars_home}/hadoop-aws-3.3.2.jar",
    f"{jars_home}/aws-java-sdk-bundle-1.12.782.jar",
    # Kyuubi
    f"{jars_home}/kyuubi/externals/engines/spark/kyuubi-spark-sql-engine_2.12-1.10.0.jar",
    f"{jars_home}/kyuubi/externals/engines/spark/kyuubi-common_2.12-1.10.0.jar"
]

# Convert to comma-separated string
jars = ",".join(jars_list)

# Create SparkSession using the builder pattern
builder = (SparkSession.builder
           .appName("DeltaExample")
           .master("local[*]")
           # Add debug configurations
           .config("spark.hadoop.fs.s3a.connection.maximum", "1")
           .config("spark.hadoop.fs.s3a.attempts.maximum", "1")
           .config("spark.hadoop.fs.s3a.connection.timeout", "5000")
           .config("spark.hadoop.fs.s3a.impl.disable.cache", "true")
           .config("spark.hadoop.fs.s3a.debug.detailed.exceptions", "true")
           # Add jars directly
           .config("spark.jars", jars)
           .config("spark.driver.extraClassPath", jars)
           .config("spark.executor.extraClassPath", jars)
           # Delta Lake configurations
           .config("spark.sql.extensions", "io.delta.sql.DeltaSparkSessionExtension")
           .config("spark.sql.catalog.spark_catalog", "org.apache.spark.sql.delta.catalog.DeltaCatalog")
           # S3/MinIO configurations
           .config("spark.hadoop.fs.s3a.access.key", "minioadmin")
           .config("spark.hadoop.fs.s3a.secret.key", "minioadmin")
           .config("spark.hadoop.fs.s3a.endpoint", "http://minio:9000")
           .config("spark.hadoop.fs.s3a.path.style.access", "true")
           .config("spark.hadoop.fs.s3a.impl", "org.apache.hadoop.fs.s3a.S3AFileSystem")
           .config("spark.hadoop.fs.s3a.aws.credentials.provider", "org.apache.hadoop.fs.s3a.SimpleAWSCredentialsProvider")
           .config("spark.hadoop.fs.s3a.connection.ssl.enabled", "false")
           # Additional Delta Lake configurations
           .config("spark.delta.logStore.class", "io.delta.storage.S3SingleDriverLogStore")
           .config("spark.hadoop.fs.s3a.fast.upload", "true")
           .config("spark.hadoop.fs.s3a.multipart.size", "104857600")
           .config("spark.sql.warehouse.dir", "/tmp/spark-warehouse"))

# Stop any existing session
if 'spark' in locals():
    spark.stop()


# Create the session
spark = builder.enableHiveSupport().getOrCreate()

# Initialize Delta Lake settings
# spark.sql("SET spark.databricks.delta.formatCheck.enabled=false")

# Access the SparkContext
# sc = spark.sparkContext

# Set the log level to INFO
# sc.setLogLevel("DEBUG")

# Test DataFrame
data = [(1, "John"), (2, "Jane")]
df = spark.createDataFrame(data, ["id", "name"])
df.show()

# First verify the S3 connection by listing the bucket
try:
    # Try to write to a simple parquet file first to test S3 connection
    print("Testing S3 connection with parquet write...")
    df.write.format("parquet").mode("overwrite").save("s3a://wba/test.parquet")
    print("S3 connection successful")

    print("Attempting to write Delta table...")
    df.write \
        .format("delta") \
        .mode("overwrite") \
        .option("overwriteSchema", "true") \
        .option("delta.compatibility.symlinkFormatManifest.enabled", "false") \
        .save("s3a://wba/example-table")
    print("Successfully wrote Delta table")

except Exception as e:
    print(f"Error: {str(e)}")
    print("\nTrying local filesystem instead...")
    try:
        local_path = "/tmp/test-delta-table"
        df.write \
            .format("delta") \
            .mode("overwrite") \
            .option("overwriteSchema", "true") \
            .save(local_path)
        print(f"Successfully wrote to {local_path}")
    except Exception as local_e:
        print(f"Error writing to local filesystem: {str(local_e)}")

25/02/23 14:37:42 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
25/02/23 14:37:42 WARN DependencyUtils: Local jar /workspace/delta-jars/delta-spark_2.12-3.2.1.jar does not exist, skipping.
25/02/23 14:37:42 WARN DependencyUtils: Local jar /workspace/delta-jars/delta-storage-3.2.1.jar does not exist, skipping.
25/02/23 14:37:42 WARN DependencyUtils: Local jar /workspace/delta-jars/kyuubi/externals/engines/spark/kyuubi-spark-sql-engine_2.12-1.10.0.jar does not exist, skipping.
25/02/23 14:37:42 WARN DependencyUtils: Local jar /workspace/delta-jars/kyuubi/externals/engines/spark/kyuubi-common_2.12-1.10.0.jar does not exist, skipping.
25/02/23 14:37:43 INFO SparkContext: Running Spark version 3.5.4
25/02/23 14:37:43 INFO SparkContext: OS info Linux, 5.15.167.4-microsoft-standard-WSL2, amd64
25/02/23 14:37:43 INFO SparkContext: Java version 17.0.14
25/02/23 14:37:43 INFO ResourceUtils: ===========================

+---+----+
| id|name|
+---+----+
|  1|John|
|  2|Jane|
+---+----+

Testing S3 connection with parquet write...


25/02/23 14:37:58 WARN MetricsConfig: Cannot locate configuration: tried hadoop-metrics2-s3a-file-system.properties,hadoop-metrics2.properties
25/02/23 14:37:58 INFO MetricsSystemImpl: Scheduled Metric snapshot period at 10 second(s).
25/02/23 14:37:58 INFO MetricsSystemImpl: s3a-file-system metrics system started
25/02/23 14:37:58 WARN VersionInfoUtils: The AWS SDK for Java 1.x entered maintenance mode starting July 31, 2024 and will reach end of support on December 31, 2025. For more information, see https://aws.amazon.com/blogs/developer/the-aws-sdk-for-java-1-x-is-in-maintenance-mode-effective-july-31-2024/
You can print where on the file system the AWS SDK for Java 1.x core runtime is located by setting the AWS_JAVA_V1_PRINT_LOCATION environment variable or aws.java.v1.printLocation system property to 'true'.
This message can be disabled by setting the AWS_JAVA_V1_DISABLE_DEPRECATION_ANNOUNCEMENT environment variable or aws.java.v1.disableDeprecationAnnouncement system property to

S3 connection successful
Attempting to write Delta table...


25/02/23 14:38:03 INFO DeltaLog: Creating initial snapshot without metadata, because the directory is empty
25/02/23 14:38:04 INFO DummySnapshot: [tableId=ae6ee1ae-f69f-4519-ab3c-ba66fecdc028] Created snapshot DummySnapshot(path=s3a://wba/example-table/_delta_log, version=-1, metadata=Metadata(c66624a1-d4c0-4344-94bc-1587d3bb458d,null,null,Format(parquet,Map()),null,List(),Map(),Some(1740321483918)), logSegment=LogSegment(s3a://wba/example-table/_delta_log,-1,List(),org.apache.spark.sql.delta.EmptyCheckpointProvider$@cdf4e87,-1), checksumOpt=None)
25/02/23 14:38:04 INFO DeltaLog: Creating initial snapshot without metadata, because the directory is empty
25/02/23 14:38:04 INFO DummySnapshot: [tableId=c66624a1-d4c0-4344-94bc-1587d3bb458d] Created snapshot DummySnapshot(path=s3a://wba/example-table/_delta_log, version=-1, metadata=Metadata(298540c1-3ab5-4ce1-a238-6ac9717139c1,null,null,Format(parquet,Map()),null,List(),Map(),Some(1740321484212)), logSegment=LogSegment(s3a://wba/example-ta

Successfully wrote Delta table


25/02/23 14:38:08 INFO CheckpointFileManager: Renamed temp file s3a://wba/example-table/_delta_log/.00000000000000000000.crc.87666be1-fe59-413e-96e4-c4becea6fe28.tmp to s3a://wba/example-table/_delta_log/00000000000000000000.crc
